In [1]:
from collections import deque
from queue import PriorityQueue
from load_data_old import make_growth_target_df
import pandas as pd
import numpy as np
import math
import time
from itertools import combinations

## aantekeningen
\
windowsize is erg bepalend in welke subgroups de hoogste quality measure halen (max 72 vs 250 bij 7 vs 5 windowsize) \
\
windowsize, windowoverlap en aggregate functies moeten nog als input aan beam search toegevoegd worden (voor de quality measure) \
\
we doen nu alleen equal voor classes column maar moet ook niet not equal? (dan neemt mogelijke optie wel gigantish toe (exponentieel??)) \
\
worden de splitpoints voor numerical class nu wel goed berekent en hoeveel bins is optimaal? \
\
is equalsized bins de beste optie? \
\
in de results opslaan hoe groot de subgroup is \ 
\ 
als je max gebruikt over de z_scores gaat hij dan niet altijd opzoek naar een subgroup waar een gigantische outlier een onderdeel van is?\
\


In [2]:
# Timing decorator to profile any function
def time_function(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Function {func.__name__} took {end_time - start_time:.4f} seconds")
        return result

    return wrapper

In [109]:
def gt(a, b):
    """Returns True if a is greater than b."""
    return a > b

def leeq(a, b):
    """Returns True if a is less than or equal to b."""
    return a <= b

def eq(a, b):
    """Returns True if a is equal to b."""
    return a == b

def neq(a, b):
    """Returns True if a is not equal to b."""
    return not eq(a, b)

def extract_subgroup(descriptors, data, col_index_dict):
    """Extracts a subgroup of data that matches all provided descriptors.

    Args:
        descriptors: A list of descriptors, each containing an attribute name, value, and operator.
        data: The dataset from which to extract the subgroup.
        col_index_dict: A dictionary mapping column names to their indices.

    Returns:
        A list of rows from the data that match all descriptors.
    """
    result = []
    for row in data:
        check = True
        for attribute in descriptors:
            att_name, descr_value, operator = attribute  # unpack 3 values from attribute
            att_index = col_index_dict[att_name]  # get the index for the attribute
            value = operator(row[att_index], descr_value)  # apply the operator

            if not value:  # if any descriptor does not match
                check = False
                break

        if check:  # if all descriptors match
            result.append(row)  # add the row to the result

    return result

def refin(seed, data, types, nr_bins, descr_indices, index_col_dict, col_index_dict):
    """Generates new descriptor sets by refining the seed descriptors.

    Args:
        seed: The initial set of descriptors to refine.
        data: The dataset used for extracting subgroups.
        types: The types of each column in the dataset.
        nr_bins: The number of bins to create for numeric attributes.
        descr_indices: The indices of potential descriptors.
        index_col_dict: A dictionary mapping index to column names.
        col_index_dict: A dictionary mapping column names to indices.

    Returns:
        A list of new descriptor sets created by refining the seed.
    """
    res = []
    used_descr = [col_index_dict[i[0]] for i in seed]  # get used descriptors
    not_used_indices = descr_indices[:]  # make a copy of descriptor indices
    # Filter out indices that are already used or are numeric types
    not_used_indices = [i for i in not_used_indices if i not in used_descr or types[i] == "numeric"]

    for i in not_used_indices:
        aux = list(seed)[:]  # copy the seed

        if types[i] == 'numeric':
            s = extract_subgroup(seed, data, col_index_dict)  # extract subgroup based on seed
            all_values = [float(entry[i]) for entry in s]  # get all values for the numeric attribute
            all_values = sorted(all_values)  # sort values
            n = len(all_values)
            # Create split points for binning
            split_points = [all_values[math.floor(j * (n/nr_bins))] for j in range(1, nr_bins)]
            for s in split_points:
                func1 = leeq
                func2 = gt

                # Create two new descriptors for each split point
                local0 = aux[:]
                local0.append((index_col_dict[i], s, func1))  # descriptor for less than or equal
                res.append(local0)

                local1 = aux[:]
                local1.append((index_col_dict[i], s, func2))  # descriptor for greater than
                res.append(local1)

        elif types[i] == 'binary':
            func = eq  # equality function for binary descriptors
            local0 = aux[:]
            local0.append((index_col_dict[i], 0, func))  # descriptor for value 0
            local1 = aux[:]
            local1.append((index_col_dict[i], 1, func))  # descriptor for value 1
            res.append(local0)
            res.append(local1)

        else:  # nominal attributes
            all_values = [entry[i] for entry in data]  # get all unique values
            for j in set(all_values):
                func1 = eq
                func2 = neq
                local0 = aux[:]
                local0.append((index_col_dict[i], j, func1))  # descriptor for equality
                res.append(local0)
                # local1 = aux[:]
                # local1.append((index_col_dict[i], j, func2))  # descriptor for not equality
                # res.append(local1)

    return res

def put_item_in_queue(queue, quality, descriptor, size=0, t=0):
    """Adds an item to a priority queue based on its quality.

    Args:
        queue: The priority queue to which the item will be added.
        quality: The quality measure of the item.
        descriptor: The descriptor associated with the item.
        size: The size of the subgroup represented by the descriptor.
    """
    if queue.full():  # if the queue is full
        min_quality, min_descriptor, min_size, min_t = queue.get()  # get the lowest quality item
        if min_quality >= quality:  # if the new item is not better, put the old one back
            queue.put((min_quality, min_descriptor, min_size, min_t))
        else:  # otherwise, add the new item
            queue.put((quality, descriptor, size, t))
    else:
        queue.put((quality, descriptor, size, t))  # add new item to the queue

def categorize_columns_in_order(df, att_columns):
    """Categorizes columns of a DataFrame into numeric, binary, and nominal types.

    Args:
        df: The DataFrame containing the data.
        att_columns: The columns to categorize.

    Returns:
        A list of column types corresponding to the provided attribute columns.
    """
    column_types = []  # List to store the categories in order

    for col in att_columns:
        if pd.api.types.is_numeric_dtype(df[col]):  # Check if the column is numeric
            column_types.append('numeric')
        elif df[col].nunique() == 2:  # Check for binary columns
            column_types.append('binary')
        else:  # Otherwise, treat it as nominal
            column_types.append('nominal')

    return column_types

def make_rolling_windows(growth_target, window_size):
    """Creates rolling windows for the target data.

    Args:
        growth_target: The target data for which to create rolling windows.
        window_size: The size of each window.

    Returns:
        A new array of rolling windows.
    """
    return np.lib.stride_tricks.sliding_window_view(growth_target, window_shape=window_size)[::window_size]

def quality_measure(targets_subgroup, targets_baseline,
                    aggregate_func_window=np.mean, aggregate_func=np.max):
    """Calculates a quality measure for a subgroup compared to a baseline.

    Args:
        targets_subgroup: list with timeseries where the timeseries are divided in windows (list with lists with lists)
        targets_baseline: list with baseline target values in windows (list with lists).
        aggregate_func_window: Function to aggregate over windows (default: mean).
        aggregate_func: Function to aggregate the final quality measure (default: max).

    Returns:
        A quality score representing the difference between the subgroup and baseline.
    """
    # Aggregate points in each window for each timeseries in the subgroup
    subgroup_aggregated_windows = aggregate_func_window(targets_subgroup, axis=2)

    # Calculate mean values for the baseline and subgroup for each window
    baseline_means = np.mean(targets_baseline, axis=1)
    subgroup_means = np.mean(subgroup_aggregated_windows, axis=0)

    # Calculate absolute differences between means for each window
    abs_diff_mean = np.abs(subgroup_means - baseline_means)

    # Calculate standard error of the subgroup for each window
    subgroup_std = np.std(subgroup_aggregated_windows, axis=0)
    standard_error_subgroup = subgroup_std #/ np.sqrt(len(targets_subgroup))

    # Calculate z-scores
    z_scores = np.divide(abs_diff_mean, standard_error_subgroup, where=standard_error_subgroup != 0)

    # Calculate the final quality score
    quality_score = aggregate_func(z_scores)

    return quality_score


def beam_search(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved, subgroup_size, target, types, window_size, max_subgroup_size=100000):
    """Performs beam search to identify optimal descriptors for subgroups.

    Args:
        data: The dataset to analyze.
        targets_baseline: The baseline target values for comparison.
        column_names: The names of the columns in the dataset.
        beam_width: The number of descriptors to keep at each depth level.
        beam_depth: The maximum depth of the beam search.
        nr_bins: The number of bins to create for numeric attributes.
        nr_saved: The number of best results to save.
        subgroup_size: The minimum size of a subgroup to consider.
        target: The target variable for which to evaluate subgroups.
        types: The types of each column in the dataset (e.g., numeric, binary, nominal).
        window_size: The size of the rolling window.

    Returns:
        A list of the best descriptor sets found during the search.
    """
    # Create dictionaries for indexing columns by name and vice versa
    index_col_dict = {i: col for i, col in enumerate(column_names)}
    col_index_dict = {col: i for i, col in enumerate(column_names)}
    target_ind = column_names.index(target)  # Get index of the target column
    att_indices = list(range(len(column_names)))  # Create a list of all indices
    att_indices.remove(target_ind)  # Remove the target index from attribute indices

    # Prepare data windows with rolling windows for the target variable
    data_windows = []
    for row in data:
        new_row = row[:]  # Create a copy of the row
        new_row[target_ind] = make_rolling_windows(row[target_ind], window_size)  # Apply rolling window
        data_windows.append(new_row)  # Add the new row to data_windows

    # Update the data and baseline targets to use rolling windows
    data = data_windows
    targets_baseline = make_rolling_windows(targets_baseline, window_size)

    # Initialize a deque for the beam search and a priority queue for results
    beam_queue = deque([()])  # Start with an empty seed
    results = PriorityQueue(nr_saved)  # Queue to hold the best results

    # Iterate through each depth of the beam search
    for depth in range(beam_depth):
        beam = PriorityQueue(beam_width)  # Initialize a new beam for this depth

        # While there are seeds in the beam queue
        while bool(beam_queue):
            seed = beam_queue.popleft()  # Get the next seed descriptor
            descriptor_set = refin(seed, data, types, nr_bins, att_indices, index_col_dict, col_index_dict)  # Refine descriptors based on seed

            # Evaluate each descriptor set generated
            for descriptor in descriptor_set:
                subgroup = extract_subgroup(descriptor, data, col_index_dict)  # Extract subgroup for the current descriptor
                if len(subgroup) >= subgroup_size and len(subgroup)<max_subgroup_size:  # Ensure subgroup is large enough
                    targets_subgroup = [i[target_ind] for i in subgroup]  # Extract target values for the subgroup
                    quality_result = quality_measure(targets_subgroup, targets_baseline)  # Calculate quality measure
                    put_item_in_queue(results, quality_result, tuple(descriptor), len(subgroup))  # Add to results queue
                    put_item_in_queue(beam, quality_result, tuple(descriptor))  # Add to the current beam

        # After processing the beam, update the beam queue with new combinations
        while not beam.empty():
            new_combination = beam.get()  # Get the highest quality descriptor from the beam
            new_combination = new_combination[1]  # Extract the descriptor from the tuple
            beam_queue.append(new_combination)  # Add it to the next depth of the beam search

    # Compile results into a list and reverse to have the best results first
    results_list = []
    while not results.empty():
        item = results.get()  # Get items from the results queue
        results_list.append(item)  # Add to the results list
    results_list.reverse()  # Reverse the list to have best results first

    return results_list  # Return the list of best descriptor sets


def filter_df_on_descriptors(df, descriptors):
    """Filters a DataFrame based on specified descriptors.

    Args:
        df: The DataFrame to filter.
        descriptors: A list of descriptors, each containing an attribute name, value, and operator.

    Returns:
        A filtered DataFrame where all descriptors hold true.
    """
    # Loop through each descriptor to apply filtering
    for desc in descriptors:
        # Apply the operator defined in the descriptor to filter the DataFrame
        df = df[df[desc[0]].apply(lambda x: desc[2](x, desc[1]))]  # Filter based on the descriptor
    return df  # Return the filtered DataFrame



###########################################################################################################
################# BEAM SEARCH WITH (SELF MADE) CONSTRAINT #################################################
### it is finished but not perfect and complicated, maybe finding something in the literature is better ###
###########################################################################################################



def get_all_descriptors(pq, index):
    """Retrieve all descriptors from a priority queue without altering its contents.

    Args:
        pq: A priority queue containing descriptor tuples.

    Returns:
        A list of descriptors.
    """
    temp_items = []  # Temporary list to store all items from the queue
    info = []  # List to store descriptors

    # Step 1: Retrieve all items from the queue
    while not pq.empty():
        item = pq.get()  # Get item from the queue
        temp_items.append(item)  # Store the item temporarily
        info.append(item[index])  # Extract and store the descriptor

    # Step 2: Put all items back into the queue to maintain its original state
    for item in temp_items:
        pq.put(item)

    # Step 3: Return the list of all descriptors
    return info


def are_descriptors_similar(descriptor1, pq):
    """Check if the given descriptor is similar to any descriptors in the priority queue.

    Args:
        descriptor1: The first descriptor to compare (a list of (metric, value, func) tuples).
        pq: A priority queue containing other descriptors.

    Returns:
        True if a similar descriptor is found in the queue, False otherwise.
    """
    tolerance = 0.25  # Tolerance range for numeric comparison
    desc1_dict = {metric: (value, func) for metric, value, func in descriptor1}  # Convert descriptor1 to a dictionary
    descriptor_list = get_all_descriptors(pq, 1)  # Get all descriptors from the queue

    # Iterate through each descriptor in the list
    for descriptor2 in descriptor_list:
        desc2_dict = {metric: (value, func) for metric, value, func in descriptor2}  # Convert descriptor2 to a dictionary

        # If the length of descriptors doesn't match, skip
        if len(desc1_dict) != len(desc2_dict):
            continue

        all_metrics_match = True  # Flag to track if all metrics match

        # Check each metric in descriptor1
        for metric in desc1_dict:
            if metric in desc2_dict:
                value1, func1 = desc1_dict[metric]
                value2, func2 = desc2_dict[metric]

                # Check if the functions are the same
                if func1 != func2:
                    all_metrics_match = False
                    break

                # Check if numeric values are within tolerance range
                if (not isinstance(value1, str)) and abs(value1 - value2) > abs(tolerance * value1):
                    all_metrics_match = False
                    break

                # For strings, check if values are exactly the same
                if isinstance(value1, str) and value1 != value2:
                    all_metrics_match = False
                    break
            else:
                # If metric in descriptor1 is not in descriptor2, they are not similar
                all_metrics_match = False
                break

        # If all metrics match, return True (descriptors are similar)
        if all_metrics_match:
            return True

    # If no similar descriptors are found, return False
    return False

def beam_search_with_constraint_descriptor(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved, subgroup_size, target, types, window_size, max_subgroup_size=100000):
    """Performs beam search with a constraint to avoid adding similar descriptors.

    Args:
        data: The dataset to analyze.
        targets_baseline: The baseline target values for comparison.
        column_names: The names of the columns in the dataset.
        beam_width: The number of descriptors to keep at each depth level.
        beam_depth: The maximum depth of the beam search.
        nr_bins: The number of bins to create for numeric attributes.
        nr_saved: The number of best results to save.
        subgroup_size: The minimum size of a subgroup to consider.
        target: The target variable for which to evaluate subgroups.
        types: The types of each column in the dataset (e.g., numeric, binary, nominal).
        window_size: The size of the rolling window.

    Returns:
        A list of the best descriptor sets found during the search, ensuring no similar descriptors.
    """
    # Create dictionaries for indexing columns by name and vice versa
    index_col_dict = {i: col for i, col in enumerate(column_names)}
    col_index_dict = {col: i for i, col in enumerate(column_names)}
    target_ind = column_names.index(target)  # Get index of the target column
    att_indices = list(range(len(column_names)))  # Create a list of all indices
    att_indices.remove(target_ind)  # Remove the target index from attribute indices

    # Prepare data windows with rolling windows for the target variable
    data_windows = []
    for row in data:
        new_row = row[:]  # Create a copy of the row
        new_row[target_ind] = make_rolling_windows(row[target_ind], window_size)  # Apply rolling window
        data_windows.append(new_row)  # Add the new row to data_windows

    # Update the data and baseline targets to use rolling windows
    data = data_windows
    targets_baseline = make_rolling_windows(targets_baseline, window_size)

    # Initialize a deque for the beam search and a priority queue for results
    beam_queue = deque([()])  # Start with an empty seed
    results = PriorityQueue(nr_saved)  # Queue to hold the best results
    results.put((0, [(0, 0, 0)], 0))  # Add a dummy descriptor to initialize

    # Iterate through each depth of the beam search
    for depth in range(beam_depth):
        beam = PriorityQueue(beam_width)  # Initialize a new beam for this depth

        # While there are seeds in the beam queue
        while bool(beam_queue):
            seed = beam_queue.popleft()  # Get the next seed descriptor
            descriptor_set = refin(seed, data, types, nr_bins, att_indices, index_col_dict, col_index_dict)  # Refine descriptors based on seed

            # Evaluate each descriptor set generated
            for descriptor in descriptor_set:
                subgroup = extract_subgroup(descriptor, data, col_index_dict)  # Extract subgroup for the current descriptor
                if len(subgroup) >= subgroup_size and len(subgroup) < max_subgroup_size and not are_descriptors_similar(descriptor, results):  # Ensure subgroup is large enough and descriptor is not similar
                    targets_subgroup = [i[target_ind] for i in subgroup]  # Extract target values for the subgroup
                    quality_result = quality_measure(targets_subgroup, targets_baseline)  # Calculate quality measure
                    put_item_in_queue(results, quality_result, tuple(descriptor), len(subgroup))  # Add to results queue
                    put_item_in_queue(beam, quality_result, tuple(descriptor))  # Add to the current beam

        # After processing the beam, update the beam queue with new combinations
        while not beam.empty():
            new_combination = beam.get()  # Get the highest quality descriptor from the beam
            new_combination = new_combination[1]  # Extract the descriptor from the tuple
            beam_queue.append(new_combination)  # Add it to the next depth of the beam search

    # Compile results into a list and reverse to have the best results first
    results_list = []
    while not results.empty():
        item = results.get()  # Get items from the results queue
        results_list.append(item)  # Add to the results list
    results_list.reverse()  # Reverse the list to have best results first

    return results_list  # Return the list of best descriptor sets

def extract_subgroup_index(descriptors, data, col_index_dict):
    """Extracts a subgroup of data that matches all provided descriptors.

    Args:
        descriptors: A list of descriptors, each containing an attribute name, value, and operator.
        data: The dataset from which to extract the subgroup.
        col_index_dict: A dictionary mapping column names to their indices.

    Returns:
        A list of rows from the data that match all descriptors.
    """
    result = []
    index_list = []
    for i, row in enumerate(data):
        check = True
        for attribute in descriptors:
            att_name, descr_value, operator = attribute  # unpack 3 values from attribute
            att_index = col_index_dict[att_name]  # get the index for the attribute
            value = operator(row[att_index], descr_value)  # apply the operator
    
            if not value:  # if any descriptor does not match
                check = False
                break
    
        if check:  # if all descriptors match
            result.append(row)  # add the row to the result
            index_list.append(i)
    
    return result, index_list

def beam_search_with_constraint_cover(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved, subgroup_size, target, types, window_size, max_subgroup_size=100000):
    """Performs beam search with a constraint to avoid adding similar descriptors.

    Args:
        data: The dataset to analyze.
        targets_baseline: The baseline target values for comparison.
        column_names: The names of the columns in the dataset.
        beam_width: The number of descriptors to keep at each depth level.
        beam_depth: The maximum depth of the beam search.
        nr_bins: The number of bins to create for numeric attributes.
        nr_saved: The number of best results to save.
        subgroup_size: The minimum size of a subgroup to consider.
        target: The target variable for which to evaluate subgroups.
        types: The types of each column in the dataset (e.g., numeric, binary, nominal).
        window_size: The size of the rolling window.

    Returns:
        A list of the best descriptor sets found during the search, ensuring no similar descriptors.
    """
    # Create dictionaries for indexing columns by name and vice versa
    index_col_dict = {i: col for i, col in enumerate(column_names)}
    col_index_dict = {col: i for i, col in enumerate(column_names)}
    target_ind = column_names.index(target)  # Get index of the target column
    att_indices = list(range(len(column_names)))  # Create a list of all indices
    att_indices.remove(target_ind)  # Remove the target index from attribute indices

    # Prepare data windows with rolling windows for the target variable
    data_windows = []
    for row in data:
        new_row = row[:]  # Create a copy of the row
        new_row[target_ind] = make_rolling_windows(row[target_ind], window_size)  # Apply rolling window
        data_windows.append(new_row)  # Add the new row to data_windows

    # Update the data and baseline targets to use rolling windows
    data = data_windows
    targets_baseline = make_rolling_windows(targets_baseline, window_size)

    # Initialize a deque for the beam search and a priority queue for results
    beam_queue = deque([()])  # Start with an empty seed
    results = PriorityQueue(nr_saved)  # Queue to hold the best results
    results.put((0, [(0, 0, 0)], [-1],0))  # Add a dummy descriptor to initialize
    cover_weight = 0.5
    max_quality = 1
    # Iterate through each depth of the beam search
    for depth in range(beam_depth):
        beam = PriorityQueue(beam_width)  # Initialize a new beam for this depth

        # While there are seeds in the beam queue
        while bool(beam_queue):
            seed = beam_queue.popleft()  # Get the next seed descriptor
            descriptor_set = refin(seed, data, types, nr_bins, att_indices, index_col_dict, col_index_dict)  # Refine descriptors based on seed

            # Evaluate each descriptor set generated
            for descriptor in descriptor_set:
                subgroup, index_list = extract_subgroup_index(descriptor, data, col_index_dict)  # Extract subgroup for the current descriptor
                if len(subgroup) >= subgroup_size and len(subgroup) < max_subgroup_size:  # Ensure subgroup is large enough and descriptor is not similar
                    targets_subgroup = [i[target_ind] for i in subgroup]  # Extract target values for the subgroup
                    quality_result = quality_measure(targets_subgroup, targets_baseline)  # Calculate quality measure
                    cover_score = calc_cover_score(index_list, results)
                    final_score = (cover_weight * (cover_score*quality_result)) + ((1-cover_weight) * quality_result)
                    # print(descriptor)
                    # print(final_score, quality_result, cover_score)
                    # print('\n')
                    put_item_in_queue(results, final_score, tuple(descriptor), index_list, quality_result)  # Add to results queue
                    put_item_in_queue(beam, final_score, tuple(descriptor))  # Add to the current beam

        # After processing the beam, update the beam queue with new combinations
        while not beam.empty():
            new_combination = beam.get()  # Get the highest quality descriptor from the beam
            new_combination = new_combination[1]  # Extract the descriptor from the tuple
            beam_queue.append(new_combination)  # Add it to the next depth of the beam search

    # Compile results into a list and reverse to have the best results first
    results_list = []
    while not results.empty():
        item = results.get()  # Get items from the results queue
        results_list.append(item)  # Add to the results list
    results_list.reverse()  # Reverse the list to have best results first

    return results_list  # Return the list of best descriptor sets

def calc_cover_score(new_index_list, results_pq, alpha):
    all_result_index_list = get_all_descriptors(results_pq, 2)
    all_cover_scores = []
    for result_index_list in all_result_index_list:
        covered_tuples = [t for t in new_index_list if t in result_index_list]
        # if not new_index_list:  # If subgroup G is empty
        #     cover_score =  0
        # else:
        cover_score = alpha ** len(covered_tuples)
        all_cover_scores.append(cover_score)
        print(all_cover_scores, len(covered_tuples))
    return min(all_cover_scores)



def calc_cover_score(new_index_list, results_pq, alpha):
    all_result_index_list = get_all_descriptors(results_pq, 2)
    alpha_scores = []
    for index in new_index_list:
        count_index = sum(sublist.count(index) for sublist in all_result_index_list)
        alpha_scores.append(alpha ** count_index)
    cover_score = sum(alpha_scores) / len(new_index_list)
    return cover_score


def calc_cover_score(new_index_list, results_pq):
    all_result_index_list = get_all_descriptors(results_pq, 2)
    new_index_set = set(new_index_list)  # Convert to set for faster lookups
    size_new_index = len(new_index_set)   # Pre-calculate size
    all_cover_scores = []

    for result_index_list in all_result_index_list:
        result_index_set = set(result_index_list)  # Convert to set
        covered_tuples = new_index_set.intersection(result_index_set)  # Get intersection
        size_result_index = len(result_index_set)  # Size of result index list
        size = max(size_new_index, size_result_index)  # Max size for cover score calculation
        cover_score = len(covered_tuples) / size  # Calculate cover score
        all_cover_scores.append(cover_score)

    return 1 - max(all_cover_scores)


def beam_search_with_constraint_descriptor2(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved, subgroup_size, target, types, window_size, max_subgroup_size=100000):
    """Performs beam search with a constraint to avoid adding similar descriptors.

    Args:
        data: The dataset to analyze.
        targets_baseline: The baseline target values for comparison.
        column_names: The names of the columns in the dataset.
        beam_width: The number of descriptors to keep at each depth level.
        beam_depth: The maximum depth of the beam search.
        nr_bins: The number of bins to create for numeric attributes.
        nr_saved: The number of best results to save.
        subgroup_size: The minimum size of a subgroup to consider.
        target: The target variable for which to evaluate subgroups.
        types: The types of each column in the dataset (e.g., numeric, binary, nominal).
        window_size: The size of the rolling window.

    Returns:
        A list of the best descriptor sets found during the search, ensuring no similar descriptors.
    """
    # Create dictionaries for indexing columns by name and vice versa
    index_col_dict = {i: col for i, col in enumerate(column_names)}
    col_index_dict = {col: i for i, col in enumerate(column_names)}
    target_ind = column_names.index(target)  # Get index of the target column
    att_indices = list(range(len(column_names)))  # Create a list of all indices
    att_indices.remove(target_ind)  # Remove the target index from attribute indices

    # Prepare data windows with rolling windows for the target variable
    data_windows = []
    for row in data:
        new_row = row[:]  # Create a copy of the row
        new_row[target_ind] = make_rolling_windows(row[target_ind], window_size)  # Apply rolling window
        data_windows.append(new_row)  # Add the new row to data_windows

    # Update the data and baseline targets to use rolling windows
    data = data_windows
    targets_baseline = make_rolling_windows(targets_baseline, window_size)

    # Initialize a deque for the beam search and a priority queue for results
    beam_queue = deque([()])  # Start with an empty seed
    results = PriorityQueue(nr_saved)  # Queue to hold the best results
    results.put((0, [(0, 0, 0)], 0, 0))  # Add a dummy descriptor to initialize

    # Iterate through each depth of the beam search
    for depth in range(beam_depth):
        beam = PriorityQueue(beam_width)  # Initialize a new beam for this depth

        # While there are seeds in the beam queue
        while bool(beam_queue):
            seed = beam_queue.popleft()  # Get the next seed descriptor
            descriptor_set = refin(seed, data, types, nr_bins, att_indices, index_col_dict, col_index_dict)  # Refine descriptors based on seed

            # Evaluate each descriptor set generated
            for descriptor in descriptor_set:
                subgroup = extract_subgroup(descriptor, data, col_index_dict)  # Extract subgroup for the current descriptor
                if len(subgroup) >= subgroup_size and len(subgroup) < max_subgroup_size:  # Ensure subgroup is large enough and descriptor is not similar
                    targets_subgroup = [i[target_ind] for i in subgroup]  # Extract target values for the subgroup
                    quality_result = quality_measure(targets_subgroup, targets_baseline)  # Calculate quality measure
                    if not descriptors_similar_paper3(quality_result, descriptor, results):
                        put_item_in_queue(results, quality_result, tuple(descriptor), len(subgroup))  # Add to results queue
                        put_item_in_queue(beam, quality_result, tuple(descriptor))  # Add to the current beam

        # After processing the beam, update the beam queue with new combinations
        while not beam.empty():
            new_combination = beam.get()  # Get the highest quality descriptor from the beam
            new_combination = new_combination[1]  # Extract the descriptor from the tuple
            beam_queue.append(new_combination)  # Add it to the next depth of the beam search

    dominance_pruning(results, subgroup_size, col_index_dict, targets_baseline, data, target_ind)
    # Compile results into a list and reverse to have the best results first
    results_list = []
    while not results.empty():
        item = results.get()  # Get items from the results queue
        results_list.append(item)  # Add to the results list
    results_list.reverse()  # Reverse the list to have best results first

    return results_list  # Return the list of best descriptor sets

def are_descriptors_similar2(quality, descriptor1, pq):
    """Check if the given descriptor is similar to any descriptors in the priority queue.

    Args:
        descriptor1: The first descriptor to compare (a list of (metric, value, func) tuples).
        pq: A priority queue containing other descriptors.

    Returns:
        True if a similar descriptor is found in the queue, False otherwise.
    """
    tolerance = 0.1 # Tolerance range for numeric comparison
    desc1_dict = {metric: (value, func) for metric, value, func in descriptor1}  # Convert descriptor1 to a dictionary
    descriptor_list = get_all_descriptors(pq, 1)
    quality_list = get_all_descriptors(pq, 0)
    #print(quality_list)
    quality_diff = min([abs(quality - q) for q in quality_list])
    if quality_diff > 5:
        return False
    match_count=0
    # Iterate through each descriptor in the list
    for descriptor2 in descriptor_list:
        desc2_dict = {metric: (value, func) for metric, value, func in descriptor2}  # Convert descriptor2 to a dictionary

        # If the length of descriptors doesn't match, skip
        if len(desc1_dict) != len(desc2_dict):
            continue

        all_metrics_match = True  # Flag to track if all metrics match

        # Check each metric in descriptor1
        for metric in desc1_dict:
            if match_count>=2:
                return True
            if metric in desc2_dict:
                value1, func1 = desc1_dict[metric]
                value2, func2 = desc2_dict[metric]

                # Check if the functions are the same
                if func1 != func2:
                    all_metrics_match = False
                    continue

                # Check if numeric values are within tolerance range
                if (not isinstance(value1, str)) and abs(value1 - value2) > abs(tolerance * value1):
                    all_metrics_match = False
                    continue

                # For strings, check if values are exactly the same
                if isinstance(value1, str) and value1 != value2:
                    all_metrics_match = False
                    continue
                else:
                    match_count+=1
                    all_metrics_match=True
            else:
                # If metric in descriptor1 is not in descriptor2, they are not similar
                all_metrics_match = False
                continue

        # If all metrics match, return True (descriptors are similar)
        if all_metrics_match and match_count>=2:
            return True

    # If no similar descriptors are found, return False
    return False

def descriptors_similar_paper(quality, descriptor1, pq):
    # Remove the single descriptor early exit if needed
    if len(descriptor1) == 1:
        return False

    tolerance = 0.1  # 10% tolerance
    desc1_dict = {metric: (value, func) for metric, value, func in descriptor1}
    descriptor_list = get_all_descriptors(pq, 1)
    quality_list = get_all_descriptors(pq, 0)
    if min(abs(quality - q) for q in quality_list) > 0.5:
        return False


    # Compare against each descriptor in the queue
    for descriptor2 in descriptor_list:
        desc2_dict = {metric: (value, func) for metric, value, func in descriptor2}

        match_count = 0
        total_conditions = len(desc1_dict)-1

        for metric, (value1, func1) in desc1_dict.items():
            if metric not in desc2_dict:
                continue  # If a metric is missing, no need to continue

            value2, func2 = desc2_dict[metric]

            # Function mismatch, skip this descriptor
            if func1 != func2:
                continue

            # Numeric comparison within tolerance
            if isinstance(value1, (int, float)) and isinstance(value2, (int, float)):
                relative_tolerance = tolerance * max(abs(value1), abs(value2))
                if abs(value1 - value2) > relative_tolerance:
                    continue  # Values out of tolerance range

            # String comparison for non-numeric values
            elif isinstance(value1, str) and value1 != value2:
                continue  # String values don't match

            match_count += 1

            # Require all conditions to match
            if match_count == total_conditions:
                return True

    return False

def descriptors_similar_paper2(quality, descriptor1, pq):
 

    # if len(descriptor1) == 1:
    #     return False

    tolerance = 0.1
    desc1_dict = {metric: (value, func) for metric, value, func in descriptor1}
    descriptor_list = get_all_descriptors(pq, 1)
    quality_list = get_all_descriptors(pq, 0)
    #print(descriptor_list)
    # Early exit if quality difference exceeds threshold
    # if min(abs(quality - q) for q in quality_list) > 1000:
    #     return False
    try:
        if desc1_dict['country'][0] =='Japan':
            print(desc1_dict)
    except:
        pass
    # Compare against each descriptor in the queue
    for descriptor2 in descriptor_list:

        desc2_dict = {metric: (value, func) for metric, value, func in descriptor2}
        #print(desc2_dict)
        match_count = 0
        try:
            if desc1_dict['country'][0] =='Japan':
                print(desc2_dict)
        except:
            pass
        for metric, (value1, func1) in desc1_dict.items():
            if metric not in desc2_dict:
                continue  # If a metric is missing, no need to continue

            value2, func2 = desc2_dict[metric]
            
            # Function mismatch, skip this descriptor
            if func1 != func2:
                continue

            # Numeric comparison within tolerance
            if isinstance(value1, (int, float)) and isinstance(value2, (int, float)):
                if value1 != value2:
                    continue  # Values out of tolerance range

            # String comparison for non-numeric values
            elif isinstance(value1, str) and value1 != value2:
                continue  # String values don't match

            match_count += 1

            # Early exit when all metrics except 1 match
            
            if match_count >= len(descriptor1)-1:
                try:
                    if desc1_dict['country'][0] =='Japan':
                        print('true \n')
                except:
                    pass
                    
                
                return True
    
    try:
        if desc1_dict['country'][0] =='Japan':
            print('false',len(descriptor_list), '\n')
    except:
        pass
    return False

@time_function
def dominance_pruning(pq, subgroup_size, col_index_dict, targets_baseline, data, target_ind):
    descriptor_list = get_all_descriptors(pq, 1)
    quality_list = get_all_descriptors(pq, 0)
    for org_quality, org_descriptor in zip(quality_list, descriptor_list):
        
        for cc in range(len(org_descriptor)-1):  # Use a while loop as the length of the list may change during pruning
            
            temp_subgroup = org_descriptor[:cc] + org_descriptor[cc+1:]  # Remove the current condition
            subgroup = extract_subgroup(temp_subgroup, data, col_index_dict)  # Extract subgroup for the current descriptor
            if len(subgroup) >= subgroup_size:  # Ensure subgroup is large enough and descriptor is not similar
                targets_subgroup = [j[target_ind] for j in subgroup]  # Extract target values for the subgroup
                quality_result = quality_measure(targets_subgroup, targets_baseline)  # Calculate quality measure
                if not descriptors_similar_paper3(quality_result, temp_subgroup, pq) and quality_result > org_quality:
                    put_item_in_queue(pq, quality_result, tuple(temp_subgroup), len(subgroup))


def descriptors_similar_paper3(quality, descriptor1, pq):

    if len(descriptor1) == 1:
        return False

    desc1_dict = {metric: (value, func) for metric, value, func in descriptor1}
    descriptor_list = get_all_descriptors(pq, 1)
    quality_list = get_all_descriptors(pq, 0)

    # Early exit if quality difference exceeds threshold
    if min(abs(quality - q) for q in quality_list) > 0.5:
        return False

    # Compare against each descriptor in the queue
    for descriptor2 in descriptor_list:

        desc2_dict = {metric: (value, func) for metric, value, func in descriptor2}

        match_count = 0

        for metric, (value1, func1) in desc1_dict.items():
            if metric not in desc2_dict:
                continue  # If a metric is missing, no need to continue

            value2, func2 = desc2_dict[metric]

            # Function mismatch, skip this descriptor
            if func1 != func2:
                continue

            # Numeric comparison within tolerance
            if isinstance(value1, (int, float)) and isinstance(value2, (int, float)):
                if value1 != value2:
                    continue  # Values out of tolerance range

            # String comparison for non-numeric values
            elif isinstance(value1, str) and value1 != value2:
                continue  # String values don't match

            match_count += 1

            # Early exit when all metrics except 1 match
            if match_count == len(desc1_dict)-1:
                return True

    return False


# Helper function to check if two metrics match with tolerance
def metrics_match(metric1, metric2):
    value1, func1 = metric1
    value2, func2 = metric2

    # If functions are different, metrics don't match
    if func1 != func2:
        return False

    # If both values are numbers, compare within tolerance
    if value1 != value2:
        return False

    # In all other cases, no match
    return True
    
def descriptors_similar_paper3(quality, descriptor1, pq):
    # If descriptor1 has only one metric, exit early
    if len(descriptor1) == 1:
        return False

    total_conditions = len(descriptor1) - 1
    descriptor_list = get_all_descriptors(pq, 1)
    quality_list = get_all_descriptors(pq, 0)

    # Early exit based on quality difference
    if min(abs(quality - q) for q in quality_list) > 0.25:
        return False

    # Compare descriptor1 against all other descriptors in pq
    for descriptor2 in descriptor_list:
        match_count = 0

        # Create a copy of descriptor2 to mark used metrics
        remaining_descriptor2 = list(descriptor2)

        # Compare each metric in descriptor1 to all available metrics in descriptor2
        for metric1_name, value1, func1 in descriptor1:
            matched = False

            # Iterate over all metrics in descriptor2 with the same name
            for i, (metric2_name, value2, func2) in enumerate(remaining_descriptor2):
                if metric1_name == metric2_name:
                    if metrics_match((value1, func1), (value2, func2)):
                        matched = True
                        # Mark this metric as used by removing it from remaining metrics
                        remaining_descriptor2.pop(i)
                        break

            if matched:
                match_count += 1

            # If all but one metrics match, descriptors are considered similar
            if match_count >= total_conditions:
                return True

    # If no matching descriptor is found, return False
    return False
def dominance_pruning(pq, subgroup_size, col_index_dict, targets_baseline, data, target_ind):
    descriptor_list = get_all_descriptors(pq, 1)
    quality_list = get_all_descriptors(pq, 0)
    for org_quality, org_descriptor in zip(quality_list, descriptor_list):

        for cc in range(len(org_descriptor)-1):  # Use a while loop as the length of the list may change during pruning

            temp_subgroup = org_descriptor[:cc] + org_descriptor[cc+1:]  # Remove the current condition
            subgroup = extract_subgroup(temp_subgroup, data, col_index_dict)  # Extract subgroup for the current descriptor
            if len(subgroup) >= subgroup_size:  # Ensure subgroup is large enough and descriptor is not similar
                targets_subgroup = [j[target_ind] for j in subgroup]  # Extract target values for the subgroup
                quality_result = quality_measure(targets_subgroup, targets_baseline)  # Calculate quality measure
                if not descriptors_similar_paper3(quality_result, temp_subgroup, pq) and quality_result > org_quality:
                    put_item_in_queue(pq, quality_result, tuple(temp_subgroup), len(subgroup))

In [4]:
stock_df = make_growth_target_df('datasets/stock_data_for_emm.pkl')

In [5]:
#stock_df.drop(['index'], inplace=True, axis=1)
stock_df

,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,fullTimeEmployees,growth_target
0,France,Aerospace & Defense,EUR,Europe/Paris,PAR,Industrials,382.0,11.822,5.750674e+07,50.783,1102.0,"[0.0, 1.17, 2.41, 0.41, 2.65, -1.39, -7.24, -4..."
1,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,32.492,650.0,"[0.0, -1.84, 17.01, 3.7, -8.24, 8.01, -3.6, 4...."
2,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,181.181,34.0,"[0.0, -7.96, -2.61, 0.89, 13.94, -4.85, -6.12,..."
3,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,68.513,232.0,"[0.0, -3.39, 0.58, -10.17, 11.33, -2.91, 2.1, ..."
4,United Kingdom,Other Precious Metals & Mining,EUR,Europe/Berlin,FRA,Basic Materials,108.0,1.402,2.123667e+08,81.212,3474.0,"[0.0, 8.5, -15.79, 5.28, 2.24, -1.94, 7.5, -1...."
...,...,...,...,...,...,...,...,...,...,...,...,...
9774,Japan,Electrical Equipment & Parts,JPY,Europe/London,LSE,Industrials,4893.0,6.287,2.079737e+10,10.654,145696.0,"[0.0, 1.08, 10.44, -11.75, -11.15, 7.96, -9.17..."
9775,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,78360.0,"[0.0, 5.82, 1.71, 7.84, -10.55, 3.58, -7.94, 2..."
9776,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,4894.0,"[0.0, 6.5, 0.0, -10.85, -1.95, -3.62, -3.95, 9..."
9777,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,1.943,1297.0,"[0.0, 0.0, 0.0, 0.0, 18.86, 0.0, 0.0, 0.0, 10...."


In [6]:
data = stock_df.values.tolist()
column_names = list(stock_df.columns)
beam_width = 10
beam_depth = 3
nr_bins = 8
nr_saved = 10
subgroup_size = len(data)*0.05
target = 'growth_target'
window_size = 5

In [7]:
target_ind = column_names.index(target)
all_time_series = [i[target_ind] for i in data]
all_time_series = np.array(all_time_series)
targets_baseline = np.mean(all_time_series, axis=0)

In [78]:
att_indices = list(range(0, len(column_names)))
att_indices.remove(target_ind)
att_columns = [column_names[i] for i in att_indices]
types = categorize_columns_in_order(stock_df, att_columns)

In [110]:
results  = beam_search_with_constraint_descriptor2(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved, subgroup_size, target, types, window_size)


In [111]:
for r in results:
    print(r[:3])
    print('\n')

(7.274882219354637, (('country', 'Japan', <function eq at 0x0000021B58D65B40>), ('enterpriseToEbitda', -1.044, <function gt at 0x0000021B58D67D00>), ('marketCap', 1977535730.11456, <function gt at 0x0000021B58D67D00>)), 491)


(6.803150219406607, (('fullTimeEmployees', 7686.0, <function gt at 0x0000021B58D67D00>), ('country', 'Japan', <function eq at 0x0000021B58D65B40>)), 616)


(6.650918653291399, (('country', 'Japan', <function eq at 0x0000021B58D65B40>), ('marketCap', 12953903104.0, <function leeq at 0x0000021B58D67EB0>), ('fullTimeEmployees', 1817.0, <function gt at 0x0000021B58D67D00>)), 575)


(6.205676446380705, (('country', 'Japan', <function eq at 0x0000021B58D65B40>), ('enterpriseToEbitda', -1.044, <function gt at 0x0000021B58D67D00>)), 656)


(5.8773758485413, (('fullTimeEmployees', 46000.0, <function gt at 0x0000021B58D67D00>), ('enterpriseToEbitda', 9.382, <function leeq at 0x0000021B58D67EB0>), ('enterpriseToEbitda', 8.486, <function leeq at 0x0000021B58D67EB0>)), 535)



In [18]:
res=0
match_count = 0
tolerance=0.1
desc1_dict = { 'country' : ('japan', eq), 'p': (-1.04, gt), 'q': (2, leeq)}
desc2_dict = { 'country' : ('japan', eq), 'p': (-1.04, gt), 'q': (10, leeq)}
for metric, (value1, func1) in desc1_dict.items():
    if metric not in desc2_dict:
        continue  # If a metric is missing, no need to continue

    value2, func2 = desc2_dict[metric]

    # Function mismatch, skip this descriptor
    if func1 != func2:
        continue

    # Numeric comparison within tolerance
    if isinstance(value1, (int, float)) and isinstance(value2, (int, float)):
        if abs(abs(value1) - abs(value2)) > abs(tolerance * value1):
            continue  # Values out of tolerance range

    # String comparison for non-numeric values
    elif isinstance(value1, str) and value1 != value2:
        print(value1, value2, '\n')
        continue  # String values don't match

    match_count += 1

    # Early exit when all metrics except 1 match
    if match_count >= 2:

        res=True
res

0

In [11]:
from beam_search_module import beam_search_with_constraint_paper

In [12]:
results = beam_search_with_constraint_paper(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins,
                                                  nr_saved, subgroup_size, target, types, window_size)

for r in results:
    print(r[:3])
    print('\n')

(6.509657278408514, (('country', 'Japan', <function eq at 0x000001FFC362EB90>), ('enterpriseToEbitda', -1.044, <function gt at 0x000001FFC362EA70>), ('enterpriseToEbitda', 3.978, <function leeq at 0x000001FFC362E9E0>)), 494)


(6.431800142548485, (('country', 'Japan', <function eq at 0x000001FFC362EB90>), ('enterpriseToEbitda', -1.044, <function gt at 0x000001FFC362EA70>), ('enterpriseToEbitda', 7.976, <function leeq at 0x000001FFC362E9E0>)), 575)


(6.205676446380705, (('country', 'Japan', <function eq at 0x000001FFC362EB90>), ('enterpriseToEbitda', -1.044, <function gt at 0x000001FFC362EA70>)), 656)


(6.035047292325761, (('country', 'Japan', <function eq at 0x000001FFC362EB90>), ('enterpriseToEbitda', -1.044, <function gt at 0x000001FFC362EA70>), ('enterpriseToEbitda', -0.356, <function gt at 0x000001FFC362EA70>)), 573)


(5.9640679219777635, (('country', 'Japan', <function eq at 0x000001FFC362EB90>), ('enterpriseToEbitda', -1.044, <function gt at 0x000001FFC362EA70>), ('enterpriseT

In [ ]:
def descriptors_similar_paper(quality, descriptor1, pq):
    # Remove the single descriptor early exit if needed
    if len(descriptor1) == 1:
        return False

    tolerance = 0.1  # 10% tolerance
    desc1_dict = {metric: (value, func) for metric, value, func in descriptor1}
    descriptor_list = get_all_descriptors(pq, 1)
    quality_list = get_all_descriptors(pq, 0)
    if min(abs(quality - q) for q in quality_list) > 0.5:
        return False


    # Compare against each descriptor in the queue
    for descriptor2 in descriptor_list:
        desc2_dict = {metric: (value, func) for metric, value, func in descriptor2}

        match_count = 0
        total_conditions = len(desc1_dict)-1

        for metric, (value1, func1) in desc1_dict.items():
            if metric not in desc2_dict:
                continue  # If a metric is missing, no need to continue

            value2, func2 = desc2_dict[metric]

            # Function mismatch, skip this descriptor
            if func1 != func2:
                continue

            # Numeric comparison within tolerance
            if isinstance(value1, (int, float)) and isinstance(value2, (int, float)):
                relative_tolerance = tolerance * max(abs(value1), abs(value2))
                if abs(value1 - value2) > relative_tolerance:
                    continue  # Values out of tolerance range

            # String comparison for non-numeric values
            elif isinstance(value1, str) and value1 != value2:
                continue  # String values don't match

            match_count += 1

            # Require all conditions to match
            if match_count == total_conditions:
                return True

    return False